In [9]:
import pandas as pd
import os
import pandas as pd

# กำหนดคอลัมน์ตามตาราง gis_vector (ยกเว้น embedding)
columns = [
    "ca_number",
    "pea_number",
    "customer_name",
    "customer_address",
    "office_code",
    "lat",
    "long",
    "billing_month"
]

def main(input_file: str):
    output_file = input_file.replace(".TXT", ".csv")    # ไฟล์ CSV ที่ต้องการบันทึก

    # อ่านไฟล์ .TXT
    df = pd.read_csv(
        input_file,
        sep="|",
        header=None,
        dtype=str,
        encoding="cp874",   # ใช้ cp874 (windows-874) ถ้าอ่านไม่ได้ลอง "tis-620"
        on_bad_lines="skip",
        engine="python"
    )

    # ตัดช่องว่างซ้ายขวาออกทุก field
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # เลือกเฉพาะคอลัมน์ที่ต้องใช้ (ตามลำดับ)
    df = df.iloc[:, [1, 0, 9, 10, 4, 2, 3, 6]]
    df.columns = columns

    # แปลง lat/long เป็นตัวเลข
    df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
    df["long"] = pd.to_numeric(df["long"], errors="coerce")

    # ลบแถวที่ไม่มี ca_number (Primary Key)
    df = df[df["ca_number"].notna() & (df["ca_number"] != "")]

    # บันทึกเป็น CSV
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"✅ Data cleaned and saved to {output_file}")

if __name__ == "__main__":
    folder_name = 'gis_data'
    print(os.listdir(folder_name))
    for file in os.listdir(folder_name):
        if file.endswith(".TXT"):
            main(os.path.join(folder_name, file))
            break

['GIS_A_202507.TXT', 'GIS_B_202507.TXT', 'GIS_C_202507.TXT', 'GIS_D_202507.TXT', 'GIS_E_202507.TXT', 'GIS_F_202507.TXT', 'GIS_G_202507.TXT', 'GIS_H_202507.TXT', 'GIS_I_202507.TXT', 'GIS_J_202507.TXT', 'GIS_K_202507.TXT']


C:\Users\502872\AppData\Local\Temp\5\ipykernel_14300\3878859204.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


✅ Data cleaned and saved to gis_data\GIS_A_202507.csv
